In [ ]:
import json
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import statistics
import visualizationutils as visutil
import statsutils

# Configuration

In [ ]:
prompt = """\
Specify the **absolute path** of the configuration file containing information about:

- the independent data file and metrics
- the dependent data file and metrics

Note: Assumptions about input files:

- independent metrics are categorical columns within independent data. This usually requires some sort \
of pre-processsing. For example, after cleaning baseline survey data and calculating the scores across \
various scales including depression scale (CES-D or BDI-II), we need to assign the category associated \
with each score ('YES', 'NO' for CES-D or 'NOT', 'LOW', 'MID', 'HIG') before using this file.
- dependent metrics do not need any further filtering by any other information within dependent data. \
e.g. in processing main sleep, dependent data only includes main sleep information. 
- dependent metrics are always averaged for each PID. This means we compare the average values per participant.

Example (find a sample config file in script-input repository): btw-sleep-baseline.json 

Tips:

- Place your configuration files in the same directory as this notebook.
- Use a different configuration file for each different analysis rather than modifying a single configuration file.
  For example, have separate files for weekly surveys and EMA surveys.

  
"""
#config_file = 'btw-sleep-baseline.json'
config_file = input(prompt)
print('using configurations specified in {}'.format(config_file))

In [ ]:
with open(config_file, 'r') as file_obj:
    config = json.load(file_obj)

independent_data_file = config['independent_data']
independent_metrics = config['independent_metrics']
dependent_data_file = config['dependent_data']
dependent_metrics = config['dependent_metrics']
institutions = config['institutions']
between_box = config['between_box']

# Setup

In [ ]:
independent = pd.read_csv(independent_data_file)
independent = independent[independent['institution'].isin(institutions)]

In [ ]:
# identify PID's in each group of a metric
independent_grouping = {institution : 
                        {metric : independent[independent['institution'] == institution][['PID', metric]].groupby(by=metric)['PID'].unique() 
                         for metric in independent_metrics} 
                        for institution in institutions}

In [ ]:
# make sure PID's in each group of a metric do not overlap
temp = independent_grouping.copy()
for institution in institutions:
    for metric in independent_metrics:
        for group in independent_grouping[institution][metric].index:
            other_pids = []
            #print('evaluating pids of', group, 'in', metric, 'of', institution)
            for other_group in independent_grouping[institution][metric].index:
                if group == other_group:
                    continue
                #print('adding pids for', other_group, 'in', metric, 'of', institution)
                other_pids.extend(independent_grouping[institution][metric][other_group])
            pids = set(independent_grouping[institution][metric][group])
            other_pids = set(other_pids)
            temp[institution][metric][group] = list(pids - other_pids)
            overlap = list(pids & other_pids)
            if(len(overlap)) > 0 :
                print('overlapping PIDs in {} of {}: {}'.format(metric, institution, overlap))
independent_grouping = temp

In [ ]:
dependent = pd.read_csv(dependent_data_file)
dependent = dependent[dependent['institution'].isin(institutions)] 

In [ ]:
# aggregate dependent data if necessary (e.g. avg sleep metrics for each PID)
dependent = dependent.groupby(['PID', 'institution'])[dependent_metrics].mean()
dependent = dependent.reset_index()
# TO-DO make sure between comparison does not rely on PID being the index

In [ ]:
# integrate a column in dependent data that associates rows of dependent data with categories of independent data
for institution in institutions:
    for metric in independent_metrics:
        for group in independent_grouping[institution][metric].index:
            dependent.loc[(dependent['PID'].isin(independent_grouping[institution][metric][group]))
                          & (dependent['institution'] == institution), 
                          metric] = group

# Assumptions

## Normality

In [ ]:
for institution in institutions:
    print('data distribution in', institution)
    visutil.distribution_graphs(independent_metrics, dependent_metrics, dependent)

In [ ]:
# TO-DO Q-Q plots of normality

In [ ]:
is_normal = {metric : dependent.groupby(by=metric).apply(statsutils.is_normally_distributed, dependent_metrics) 
             for metric in independent_metrics}


## homogeneity of Variance (HOV) 

In [ ]:
# test of HOV

# Comparisons

In [ ]:
#TO-DO

# Effect Size

In [ ]:
# TO-DO

NOTE: everything below this line is being refactored above

In [ ]:
def is_normally_distributed(data, column, threshold=0.05):
    # null hypothesis: data[column] comes from a normal distribution
    k2, p = stats.normaltest(data[column])
    if(p < threshold):
        # we reject the null hypothesis, i.e. it is unlikely for the data to come from normal distribution
        return False 
    # we maintain the null hypothesis, i.e. there is not enought evidence that data does not come from nomral distribution
    return True 

In [ ]:
def is_HOV_met(data1, data2, column, center='mean', threshold=0.05):
    # null hypothesis: data1[column] and data2[column] have equal variances
    w, p = stats.levene(data1[column], data2[column])
    if(p < threshold):
        # we reject the null hypothesis, i.e. the variances are unequal
        return False 
    # we maintain the null hypothesis, i.e. there is not enought evidence that the variances are unequal
    return True 

In [ ]:
# one-way ANOVA comparing affect ratings between participants who reported discrimination at least once and
# those who did not report discrimination at all
def between_comparison(independent_column_name, dependent_column_name, resps):
    F, p =  stats.f_oneway(resps[resps[independent_column_name] == 'YES'][dependent_column_name], 
                           resps[resps[independent_column_name] == 'NO'][dependent_column_name])
    #F, p = stats.mannwhitneyu(resps[resps['discriminated'] == 'YES'][dependent_column_name], 
    #                       resps[resps['discriminated'] == 'NO'][dependent_column_name]) # TO-DO test
    print('one-way ANOVA on {} for {}: F = {:.2f}, p = {:.3f}'.format(dependent_column_name, 
                                                                      independent_column_name,
                                                                      F, 
                                                                      p))
    print('*****************')
    return (F, p)

In [ ]:
def cohen_d(x,y):
# under the HOV assumption
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    mean_diff = np.mean(x) - np.mean(y)
    pooled_var = ((nx-1)*(np.std(x, ddof=1) ** 2) + (ny-1)*(np.std(y, ddof=1) ** 2)) / dof
    return mean_diff / np.sqrt(pooled_var)

In [ ]:
def set_box_color(bp, color):
    plt.setp(bp['boxes'], color=color)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['fliers'], color=color, marker = ".")
    plt.setp(bp['medians'], color=color)

In [ ]:
sleep_file = '/Users/yasaman/UWEXP/analysis-scripts/sensors/aggregation/results/sleep_aggregated.csv'
step_file = '/Users/yasaman/UWEXP/analysis-scripts/sensors/aggregation/results/step_aggregated.csv'
baseline_scores_file = '/Users/yasaman/UWEXP/analysis-scripts/surveys/results/baselines/baseline_scores_univ_pid_categorized.csv'
sleep_between_box = '/Users/yasaman/UWEXP/analysis-scripts/surveys/results/baselines/sleep_between.png'
step_between_box = '/Users/yasaman/UWEXP/analysis-scripts/surveys/results/baselines/step_between.png'

In [ ]:
#sleep_metrics = ['totalTimeInBed', 'totalMinutesAsleep', 'minutesAwake', 'minutesAsleep', 'efficiency']
sleep_metrics = ['totalTimeInBed', 'totalMinutesAsleep']
step_metrics = ['steps']
scales = ['CES_D_POST', 'PSS_POST']

In [ ]:
sleep = pd.read_csv(sleep_file)
step = pd.read_csv(step_file)
baseline_scores = pd.read_csv(baseline_scores_file)
baseline_scores_uw = baseline_scores[baseline_scores['institution'] == 'UW']

In [ ]:
pids_stressed = set(baseline_scores_uw[baseline_scores_uw['stressed'] == 'YES']['PID'].astype('int32').unique())
pids_not_stressed = set(baseline_scores_uw[baseline_scores_uw['stressed'] == 'NO']['PID'].astype('int32').unique())
pids_depressed = set(baseline_scores_uw[baseline_scores_uw['depressed'] == 'YES']['PID'].astype('int32').unique())
pids_not_depressed = set(baseline_scores_uw[baseline_scores_uw['depressed'] == 'NO']['PID'].astype('int32').unique())

In [ ]:
sleep_pids = set(sleep['PID'].unique())
between_sleep_avg = sleep[sleep['isMainSleep'] == True].groupby(['PID'])[sleep_metrics].mean()
between_sleep_avg.loc[pids_stressed & sleep_pids, 'stressed'] = 'YES'
between_sleep_avg.loc[pids_not_stressed & sleep_pids, 'stressed'] = 'NO'
between_sleep_avg.loc[pids_depressed & sleep_pids, 'depressed'] = 'YES'
between_sleep_avg.loc[pids_not_depressed & sleep_pids, 'depressed'] = 'NO'

In [ ]:
sleep_avg_stressed_normality = [is_normally_distributed(between_sleep_avg[between_sleep_avg['stressed'] == 'YES'], 
                                                        metric, 
                                                        0.01) for metric in sleep_metrics]
print('is sleep data normally distributed for people who have no reports of discrimination?', sleep_avg_stressed_normality)

In [ ]:
sleep_avg_not_stressed_normality = [is_normally_distributed(between_sleep_avg[between_sleep_avg['stressed'] == 'NO'], 
                                                            metric, 
                                                            0.01) for metric in sleep_metrics]
print('is sleep data normally distributed for people who have no reports of discrimination?', sleep_avg_not_stressed_normality)

In [ ]:
sleep_HOV = [is_HOV_met(between_sleep_avg[between_sleep_avg['stressed'] == 'YES'],
                        between_sleep_avg[between_sleep_avg['stressed'] == 'NO'],
                        metric,
                        'median', 
                        0.05) for metric in sleep_metrics]
print('is sleep data for people who have reported discrimination and those who have not have equal variances?', sleep_HOV)

In [ ]:
[cohen_d(between_sleep_avg[between_sleep_avg['stressed'] == 'YES'][metric], 
         between_sleep_avg[between_sleep_avg['stressed'] == 'NO'][metric]) for metric in sleep_metrics]

In [ ]:
sleep_metrics

In [ ]:
[statistics.median(between_sleep_avg[between_sleep_avg['stressed'] == 'YES'][metric]) for metric in sleep_metrics]

In [ ]:
[statistics.median(between_sleep_avg[between_sleep_avg['stressed'] == 'NO'][metric]) for metric in sleep_metrics]

In [ ]:
group_means_stressed = between_sleep_avg.groupby(['stressed'])[sleep_metrics].mean()
print(group_means_stressed.T)
mean_diff = group_means_stressed.loc['YES'] - group_means_stressed.loc['NO']
print('the difference in sleep metrics in people who are stressed')
print('NOTE: positive difference means metrics are larger in people who are stressed')
print('all time measures are in minutes')
print(mean_diff)

In [ ]:
group_means_depressed = between_sleep_avg.groupby(['depressed'])[sleep_metrics].mean()
print(group_means_depressed.T)
mean_diff = group_means_depressed.loc['YES'] - group_means_depressed.loc['NO']
print('the difference in sleep metrics in people who are depressed')
print('NOTE: positive difference means metrics are larger in people who are depressed')
print('all time measures are in minutes')
print(mean_diff)

In [ ]:
stressed_between = [between_comparison('stressed', 
                                       metric, 
                                       between_sleep_avg[~between_sleep_avg['stressed'].isna()]) for metric in sleep_metrics]

In [ ]:
depressed_between = [between_comparison('depressed', 
                                        metric, 
                                        between_sleep_avg[~between_sleep_avg['depressed'].isna()]) for metric in sleep_metrics]

In [ ]:
stressed_yes = [between_sleep_avg[between_sleep_avg['stressed'] == 'YES'][metric] for metric in sleep_metrics]
stressed_no = [between_sleep_avg[between_sleep_avg['stressed'] == 'NO'][metric] for metric in sleep_metrics]

plt.figure(figsize=(8,4))

bpl = plt.boxplot(stressed_yes, 
                  positions=np.array(range(len(sleep_metrics)))*2.0-0.4, 
                  sym='', widths=0.6)
bpr = plt.boxplot(stressed_no, 
                  positions=np.array(range(len(sleep_metrics)))*2.0+0.4, 
                  sym='', widths=0.6)
set_box_color(bpl, '#D7191C') # colors are from http://colorbrewer2.org/
set_box_color(bpr, '#2C7BB6')

# draw temporary red and blue lines and use them to create a legend
plt.plot([], c='#D7191C', label='stressed (PSS >= 20)')
plt.plot([], c='#2C7BB6', label='not stressed (PSS < 20)')
plt.legend(ncol = 2)
plt.ylabel('Sleep Metrics (in min)')
plt.xticks(range(0, len(sleep_metrics) * 2, 2), sleep_metrics)
plt.xlim(-1, len(sleep_metrics)*2-1)
plt.ylim(300, 600)
plt.tight_layout()
plt.savefig(sleep_between_box)

In [ ]:
step_pids = set(step['PID'].unique())
between_step_avg = step.groupby(['PID'])[step_metrics].mean()
between_step_avg.loc[pids_stressed & step_pids, 'stressed'] = 'YES'
between_step_avg.loc[pids_not_stressed & step_pids, 'stressed'] = 'NO'
between_step_avg.loc[pids_depressed & step_pids, 'depressed'] = 'YES'
between_step_avg.loc[pids_not_depressed & step_pids, 'depressed'] = 'NO'

In [ ]:
step_avg_stressed_normality = is_normally_distributed(between_step_avg[between_step_avg['stressed'] == 'YES'], 
                                                      'steps', 
                                                      0.01)
print('is steps data normally distributed for people who reported discrimination?', step_avg_stressed_normality)

In [ ]:
# TO-DO plot step data distribution

In [ ]:
step_avg_not_stressed_normality = is_normally_distributed(between_step_avg[between_step_avg['stressed'] == 'NO'], 
                                                          'steps', 
                                                          0.01)
print('is steps data normally distributed for people who have no reports of discrimination?', step_avg_not_stressed_normality)

In [ ]:
step_HOV = is_HOV_met(between_step_avg[between_step_avg['stressed'] == 'YES'],
                      between_step_avg[between_step_avg['stressed'] == 'NO'],
                      'steps',
                      'median', 
                      0.05)
print('is steps data for people who have reported discrimination and those who have not have equal variances?', step_HOV)

In [ ]:
cohen_d(between_step_avg[between_step_avg['stressed'] == 'YES']['steps'], 
        between_step_avg[between_step_avg['stressed'] == 'NO']['steps'])

In [ ]:
group_means_stressed = between_step_avg.groupby(['stressed'])[step_metrics].mean()
print(group_means_stressed.T)
mean_diff = group_means_stressed.loc['YES'] - group_means_stressed.loc['NO']
print('the difference in step metrics in people who are stressed')
print('NOTE: positive difference means metrics are larger in people who are stressed')
print(mean_diff)

In [ ]:
np.std(between_step_avg[between_step_avg['stressed'] == 'YES'], ddof=1)

In [ ]:
statistics.median(between_step_avg[between_step_avg['stressed'] == 'YES']['steps'])

In [ ]:
statistics.median(between_step_avg[between_step_avg['stressed'] == 'NO']['steps'])

In [ ]:
group_means_depressed = between_step_avg.groupby(['depressed'])[step_metrics].mean()
print(group_means_depressed.T)
mean_diff = group_means_depressed.loc['YES'] - group_means_depressed.loc['NO']
print('the difference in step metrics in people who are depressed')
print('NOTE: positive difference means metrics are larger in people who are depressed')
print(mean_diff)

In [ ]:
stressed_between = [between_comparison('stressed', 
                                       metric, 
                                       between_step_avg[~between_step_avg['stressed'].isna()]) for metric in step_metrics]

In [ ]:
depressed_between = [between_comparison('depressed', 
                                        metric, 
                                        between_step_avg[~between_step_avg['depressed'].isna()]) for metric in step_metrics]

In [ ]:
stressed_yes = [between_step_avg[between_step_avg['stressed'] == 'YES'][metric] for metric in step_metrics]
stressed_no = [between_step_avg[between_step_avg['stressed'] == 'NO'][metric] for metric in step_metrics]

plt.figure(figsize=(8,4))

bpl = plt.boxplot(stressed_yes, 
                  positions=np.array(range(len(step_metrics)))*2.0-0.4, 
                  sym='', widths=0.6)
bpr = plt.boxplot(stressed_no, 
                  positions=np.array(range(len(step_metrics)))*2.0+0.4, 
                  sym='', widths=0.6)
set_box_color(bpl, '#D7191C') # colors are from http://colorbrewer2.org/
set_box_color(bpr, '#2C7BB6')

# draw temporary red and blue lines and use them to create a legend
plt.plot([], c='#D7191C', label='stressed (PSS >= 20)')
plt.plot([], c='#2C7BB6', label='not stressed (PSS < 20)')
plt.legend(ncol = 2)
plt.ylabel('Daily Number of Steps')
plt.xticks(range(0, len(step_metrics) * 2, 2), step_metrics)
plt.xlim(-1, len(step_metrics)*2-1)
plt.ylim(1500, 18000)
plt.tight_layout()
step_between_box = '/Users/yasaman/UWEXP/analysis-scripts/surveys/results/baselines/step_between.png'
plt.savefig(step_between_box)